In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [2]:
weather_df = pd.read_csv('../WeatherPy/Output_CSV/WeatherPy.csv')
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,filingue,14.3521,3.3168,82.78,9,5,10.00,NE,1616958474
1,akhmim,26.5622,31.7450,66.20,24,0,16.11,EG,1616958474
2,berdigestyakh,62.0989,126.6992,-4.02,100,100,2.75,RU,1616958475
3,abha,18.2164,42.5053,64.40,72,0,0.74,SA,1616958475
4,upernavik,72.7868,-56.1549,-16.58,78,31,2.57,GL,1616958357


In [3]:
# Store latitude and longitude in locations
locations = weather_df[['Lat', 'Lng']]

# Fill NaN values and convert to float
humidity = weather_df["Humidity"]
locations.head()

,Lat,Lng
0,14.3521,3.3168
1,26.5622,31.7450
2,62.0989,126.6992
3,18.2164,42.5053
4,72.7868,-56.1549


In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
max_intensity

100

In [10]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_df.loc[(weather_df["Wind Speed"] <= 100) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temp"] >= 60) & (weather_df["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,akhmim,26.5622,31.7450,66.20,24,0,16.11,EG,1616958474
3,abha,18.2164,42.5053,64.40,72,0,0.74,SA,1616958475
24,cape town,-33.9258,18.4232,69.01,72,0,8.05,ZA,1616958485
30,sakakah,29.9697,40.2064,60.80,14,0,6.91,SA,1616958488


In [11]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
1,akhmim,EG,26.5622,31.7450,
3,abha,SA,18.2164,42.5053,
24,cape town,ZA,-33.9258,18.4232,
30,sakakah,SA,29.9697,40.2064,


In [15]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : gkey}

In [18]:
# get city name, lat, lnt from df
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
        print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 1: akhmim.
Closest hotel in akhmim is فندق الفائز.
Retrieving Results for Index 3: abha.
Closest hotel in abha is Shafa Abha Hotel.
Retrieving Results for Index 24: cape town.
Closest hotel in cape town is Atlanticview Cape Town Boutique Hotel.
Retrieving Results for Index 30: sakakah.
Closest hotel in sakakah is Alnuzl Special Hotel.
-------End of Search-------


In [19]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
1,akhmim,EG,26.5622,31.7450,فندق الفائز
3,abha,SA,18.2164,42.5053,Shafa Abha Hotel
24,cape town,ZA,-33.9258,18.4232,Atlanticview Cape Town Boutique Hotel
30,sakakah,SA,29.9697,40.2064,Alnuzl Special Hotel


In [37]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [38]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))